## 题目1：航空公司客户价值分析

### 题目描述

您是某航空公司的初级数据分析师，公司希望通过分析客户的历史飞行数据来识别高价值客户，以便进行精准营销。现在您得到了一份名为 `flight_data.csv` 的模拟数据集。该数据集包含了客户的以下信息：

- `customer_id`: 客户唯一标识符
- `flight_date`: 飞行日期（格式：YYYY-MM-DD）
- `flight_number`: 航班号
- `origin`: 出发地
- `destination`: 目的地
- `fare`: 票价（可能包含非数字字符，如 'USD'）
- `cabin_class`: 舱位等级（例如：Economy, Business, First）
- `distance_miles`: 飞行距离（英里）

请您使用 Python 对这份数据进行以下分析，以实现客户价值评估：


### 目标要求：

1.  **数据加载与概览**：
    *   加载 `flight_data.csv` 文件到 Pandas DataFrame。
    *   显示数据集的前5行、数据类型和基本统计信息。

2.  **数据清洗与预处理**：
    *   **处理 `fare` 字段**：
        *   去除 'USD ' 前缀（如果存在）。
        *   将 `fare` 列转换为数值类型。
        *   使用该列的**中位数**填充缺失值。
    *   **处理 `flight_number` 字段**：
        *   将缺失值（`None`）填充为 'UNKNOWN'。
    *   **处理 `cabin_class` 字段**：
        *   将空字符串（`''`）替换为 'Economy' (假设空字符串代表经济舱)。
    *   **处理 `distance_miles` 字段**：
        *   将小于等于 0 的值替换为该列的**平均值**。
    *   **处理 `flight_date` 字段**：
        *   确保其为 datetime 类型。

3.  **计算 RFM 指标**：
    为了评估客户价值，我们需要计算每个客户的 RFM (Recency, Frequency, Monetary) 指标。
    *   **Recency (近度)**：
        *   计算每个客户**距离最近一次飞行日期**的天数。以数据集中的最大飞行日期作为“当前日期”。
    *   **Frequency (频度)**：
        *   计算每个客户**总的飞行次数**。
    *   **Monetary (价值)**：
        *   计算每个客户**总的票价花费**。

    请将这些 RFM 指标合并到一个新的 DataFrame 中，其中包含 `customer_id`, `Recency`, `Frequency`, `Monetary`。

4.  **客户分群**：
    基于 RFM 指标，对客户进行简单的分群。
    *   将 `Recency`, `Frequency`, `Monetary` 各自划分为 3 个等级 (1 到 3，等级越高代表表现越好)。
        *   `Recency`: 值越小越好 (最近飞行)，因此等级划分规则为：最小 33% 设为 3，中间 33% 设为 2，最大 33% 设为 1。
        *   `Frequency`: 值越大越好 (飞行次数多)，因此等级划分规则为：最小 33% 设为 1，中间 33% 设为 2，最大 33% 设为 3。
        *   `Monetary`: 值越大越好 (花费金额高)，因此等级划分规则与 Frequency 相同。
    *   创建一个新的列 `RFM_Score`，其值为 `Recency_Score` * 100 + `Frequency_Score` * 10 + `Monetary_Score`。
    *   根据 `RFM_Score` 对客户进行分群，例如：
        *   `RFM_Score` >= 323: "高价值客户"
        *   `RFM_Score` >= 222 and `RFM_Score` < 323: "中价值客户"
        *   `RFM_Score` < 222: "低价值客户"
    *   统计各客户群体的数量。

### 模拟数据

为了方便练习，使用以下代码创建 `flight_data.csv` 文件：


In [8]:
import pandas as pd
import numpy as np

# 模拟数据
np.random.seed(42)
num_customers = 500
num_flights = 5000

customer_ids = [f'C_{i:04d}' for i in range(1, num_customers + 1)]
flight_dates = pd.to_datetime('2022-01-01') + pd.to_timedelta(np.random.randint(0, 365, num_flights), unit='D')
flight_numbers = np.array([f'AA{np.random.randint(100, 999)}' for _ in range(num_flights)])  # Convert to numpy array
origins = np.random.choice(['JFK', 'LAX', 'ORD', 'DFW', 'ATL', 'DEN', 'SFO', 'CLT'], num_flights)
destinations = np.random.choice(['JFK', 'LAX', 'ORD', 'DFW', 'ATL', 'DEN', 'SFO', 'CLT'], num_flights)
fares = np.random.randint(50, 2000, num_flights).astype(object)
cabin_classes = np.random.choice(['Economy', 'Business', 'First'], num_flights, p=[0.7, 0.2, 0.1])
distance_miles = np.random.randint(200, 5000, num_flights)

# Introduce some missing values and dirty data
fares[np.random.choice(num_flights, 50, replace=False)] = np.nan
fares[np.random.choice(num_flights, 30, replace=False)] = 'USD ' + fares[np.random.choice(num_flights, 30, replace=False)].astype(str)
flight_numbers[np.random.choice(num_flights, 20, replace=False)] = np.nan  # This will now work
cabin_classes[np.random.choice(num_flights, 40, replace=False)] = ''
distance_miles[np.random.choice(num_flights, 60, replace=False)] = -100

data = pd.DataFrame({
    'customer_id': np.random.choice(customer_ids, num_flights, replace=True),
    'flight_date': flight_dates,
    'flight_number': flight_numbers,
    'origin': origins,
    'destination': destinations,
    'fare': fares,
    'cabin_class': cabin_classes,
    'distance_miles': distance_miles
})

data.to_csv(r'csv\flight_data.csv', index=False)
print("flight_data.csv created successfully!")

flight_data.csv created successfully!


### 答题

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

1.  **数据加载与概览**：
    *   加载 `flight_data.csv` 文件到 Pandas DataFrame。
    *   显示数据集的前5行、数据类型和基本统计信息。

In [31]:
df = pd.read_csv(r"csv\flight_data.csv")

print(f"数据前 5 行:\n{df.head()}\n")
print(f"数据类型:\n{df.dtypes}\n")
print(f"数据基本统计:\n{df.describe()}\n")


数据前 5 行:
  customer_id flight_date flight_number origin destination  fare cabin_class  \
0      C_0335  2022-04-13         AA961    ORD         JFK   443     Economy   
1      C_0147  2022-12-15         AA667    JFK         JFK  1859     Economy   
2      C_0102  2022-09-28         AA159    ORD         JFK   839    Business   
3      C_0288  2022-04-17         AA749    ORD         DEN   644       First   
4      C_0313  2022-03-13         AA138    SFO         SFO   107     Economy   

   distance_miles  
0            4980  
1            4856  
2            4418  
3            2865  
4             615  

数据类型:
customer_id       object
flight_date       object
flight_number     object
origin            object
destination       object
fare              object
cabin_class       object
distance_miles     int64
dtype: object

数据基本统计:
       distance_miles
count     5000.000000
mean      2554.478400
std       1395.315114
min       -100.000000
25%       1354.750000
50%       2590.000000
75%   

2.  **数据清洗与预处理**：
    *   **处理 `fare` 字段**：
        *   去除 'USD ' 前缀（如果存在）。
        *   将 `fare` 列转换为数值类型。
        *   使用该列的**中位数**填充缺失值。
    *   **处理 `flight_number` 字段**：
        *   将缺失值（`None`）填充为 'UNKNOWN'。
    *   **处理 `cabin_class` 字段**：
        *   将空字符串（`''`）替换为 'Economy' (假设空字符串代表经济舱)。
    *   **处理 `distance_miles` 字段**：
        *   将小于等于 0 的值替换为该列的**平均值**。
    *   **处理 `flight_date` 字段**：
        *   确保其为 datetime 类型。

In [32]:
df['fare'] = df['fare'].replace('USD ', "", regex=True).str.strip().astype(float)

df['fare'] = df['fare'].fillna(df['fare'].median())

In [33]:
df['flight_number'] = df['flight_number'].fillna("UNKNOWN")

In [36]:
df['cabin_class'] = df['cabin_class'].replace('', 'Economy', regex=True)

In [53]:
mean_distance = df[df['distance_miles'] > 0]['distance_miles'].mean()
# 推荐使用
# mean_distance = df.loc[df['distance_miles'] > 0, 'distance_miles'].mean()

df['distance_miles'] = df['distance_miles'].apply(lambda x: x if x > 0 else mean_distance)
print(f"distance_miles 异常值处理完成，小于等于0的值替换为平均值: {mean_distance:.2f}")

distance_miles 异常值处理完成，小于等于0的值替换为平均值: 2586.72


3.  **计算 RFM 指标**：
    为了评估客户价值，我们需要计算每个客户的 RFM (Recency, Frequency, Monetary) 指标。
    *   **Recency (近度)**：Recency衡量的是客户最近一次飞行距离现在有多久
        *   计算每个客户**距离最近一次飞行日期**的天数。以数据集中的最大飞行日期作为“当前日期”。
    *   **Frequency (频度)**：
        *   计算每个客户**总的飞行次数**。
    *   **Monetary (价值)**：
        *   计算每个客户**总的票价花费**。

    请将这些 RFM 指标合并到一个新的 DataFrame 中，其中包含 `customer_id`, `Recency`, `Frequency`, `Monetary`。

In [61]:
current_date = df['flight_date'].max()

def recency(data):

    global current_date
    max_date = data.max()
    return (current_date - max_date).days


# df.groupby('customer_id').agg({
#     "flight_date": recency,
#     "flight_number": "count",
#     "fare": "sum"
# })

df_rfm = df.groupby('customer_id').agg(R=('flight_date', recency),
                              F=('flight_number', 'count'),
                              M=('fare', 'sum')
                              )

print(df_rfm)



              R   F        M
customer_id                 
C_0001       40  11   9879.0
C_0002       28   6   5815.0
C_0003        1  12  10200.0
C_0004       55   4   3119.0
C_0005       24   9  10639.0
...          ..  ..      ...
C_0496       90   4   4774.0
C_0497       50  15  12357.0
C_0498       50  13  11225.0
C_0499       10  14  12194.0
C_0500       37  12  14271.0

[500 rows x 3 columns]


4.  **客户分群**：
    基于 RFM 指标，对客户进行简单的分群。
    *   将 `Recency`, `Frequency`, `Monetary` 各自划分为 3 个等级 (1 到 3，等级越高代表表现越好)。
        *   `Recency`: 值越小越好 (最近飞行)，因此等级划分规则为：最小 33% 设为 3，中间 33% 设为 2，最大 33% 设为 1。
        *   `Frequency`: 值越大越好 (飞行次数多)，因此等级划分规则为：最小 33% 设为 1，中间 33% 设为 2，最大 33% 设为 3。
        *   `Monetary`: 值越大越好 (花费金额高)，因此等级划分规则与 Frequency 相同。
    *   创建一个新的列 `RFM_Score`，其值为 `Recency_Score` * 100 + `Frequency_Score` * 10 + `Monetary_Score`。
    *   根据 `RFM_Score` 对客户进行分群，例如：
        *   `RFM_Score` >= 323: "高价值客户"
        *   `RFM_Score` >= 222 and `RFM_Score` < 323: "中价值客户"
        *   `RFM_Score` < 222: "低价值客户"
    *   统计各客户群体的数量。

In [63]:
df_rfm['R_score'] = pd.qcut(df_rfm['R'], q=3, labels=[3, 2, 1], duplicates='drop').astype(int)

df_rfm['F_score'] = pd.qcut(df_rfm['F'], q=3, labels=[1, 2, 3], duplicates='drop').astype(int)
df_rfm['M_score'] = pd.qcut(df_rfm['M'], q=3, labels=[1, 2, 3], duplicates='drop').astype(int)

df_rfm['RFM_Score'] = df_rfm['R_score'] * 100 + df_rfm['F_score'] * 10 + df_rfm['M_score']
print(df_rfm)


              R   F        M  R_score  F_score  M_score  RFM_Score
customer_id                                                       
C_0001       40  11   9879.0        2        2        2        222
C_0002       28   6   5815.0        2        1        1        211
C_0003        1  12  10200.0        3        3        2        332
C_0004       55   4   3119.0        1        1        1        111
C_0005       24   9  10639.0        2        2        2        222
...          ..  ..      ...      ...      ...      ...        ...
C_0496       90   4   4774.0        1        1        1        111
C_0497       50  15  12357.0        1        3        3        133
C_0498       50  13  11225.0        1        3        2        132
C_0499       10  14  12194.0        3        3        3        333
C_0500       37  12  14271.0        2        3        3        233

[500 rows x 7 columns]


In [66]:
def uer_level(rfm_score):
    if rfm_score >= 333:
        return '高价值客户'
    elif 222 <= rfm_score < 333:
        return '中价值客户'
    else:
        return '低价值客户'
    
df_rfm['User_Level'] = df_rfm['RFM_Score'].apply(uer_level)
print(df_rfm)

              R   F        M  R_score  F_score  M_score  RFM_Score User_Level
customer_id                                                                  
C_0001       40  11   9879.0        2        2        2        222      中价值客户
C_0002       28   6   5815.0        2        1        1        211      低价值客户
C_0003        1  12  10200.0        3        3        2        332      中价值客户
C_0004       55   4   3119.0        1        1        1        111      低价值客户
C_0005       24   9  10639.0        2        2        2        222      中价值客户
...          ..  ..      ...      ...      ...      ...        ...        ...
C_0496       90   4   4774.0        1        1        1        111      低价值客户
C_0497       50  15  12357.0        1        3        3        133      低价值客户
C_0498       50  13  11225.0        1        3        2        132      低价值客户
C_0499       10  14  12194.0        3        3        3        333      高价值客户
C_0500       37  12  14271.0        2        3        3        2

# 题目2：Word 文档自动化处理 - 项目报告模板填充

### 题目描述：

你是一家大型咨询公司的实习数据分析师。

公司有一份标准的项目报告模板（Project_Report_Template.docx），这份模板包含一些占位符，需要根据具体的项目信息进行替换。

你的任务是编写一个 Python 脚本，读取这份模板，根据提供的项目数据，自动填充并生成一份新的、完整的项目报告。

目标要求：
- 读取模板并替换：
    1. 读取 Project_Report_Template.docx。
    2. 遍历文档中的所有段落（Paragraph）和表格（Table）单元格，查找并替换 placeholder_map 中定义的所有占位符。
    3. 替换占位符：
        - 若占位符出现在段落文本中，直接替换。
        - 若占位符出现在表格单元格中，需要先获取单元格的文本，替换后再设置回单元格。
    4. 保留格式：在替换文本时，要注意保留原段落或单元格的格式，包括字体、大小、加粗、斜体等。

生成新文档：将填充完毕的报告保存为 Filled_Project_Report.docx。

模拟数据

In [ ]:
project_data = {
    "COMPANY_NAME": "ABC 数据咨询",
    "PROJECT_Slogan": "数据驱动未来", # 这是一个额外信息，需要您自行决定处理方式或忽略
    "PROJECT_TITLE": "电商客户行为分析报告",
    "PROJECT_MANAGER": "张三",
    "REPORT_DATE": "2023-10-26",
    "CUSTOMER_NAME": "XYZ 电商平台",
}


analysis_data = {
    "TOTAL_ORDERS": 12586,
    "AVERAGE_ORDER_AMOUNT": 345.78,
    "HIGH_VALUE_CUSTOMER_PERCENTAGE": 25.3
}

recommendations_text = "通过优化商品推荐算法和提升用户互动，可以进一步提高客户留存率和复购率。"

placeholder_map = {
    "[公司名称]": project_data["COMPANY_NAME"],
    "[PROJECT_NAME]": project_data["PROJECT_TITLE"],
    "[PROJECT_MANAGER]": project_data["PROJECT_MANAGER"],
    "[REPORT_DATE]": project_data["REPORT_DATE"],
    "[CUSTOMER_NAME]": project_data["CUSTOMER_NAME"],
    "[PROJECT_PURPOSE]": "深入理解客户购物习惯，识别高价值客户群体，并为精准营销提供数据支持", # 这个占位符在模板中有，但未在project_data中
    "[TOTAL_ORDERS]": analysis_data["TOTAL_ORDERS"],
    "[AVERAGE_ORDER_AMOUNT]": f"{analysis_data['AVERAGE_ORDER_AMOUNT']:.2f} 元", # 格式化输出
    "[HIGH_VALUE_CUSTOMER_PERCENTAGE]": analysis_data["HIGH_VALUE_CUSTOMER_PERCENTAGE"],
    "[RECOMMENDATIONS]": recommendations_text
}


读取 Project_Report_Template.docx

In [3]:
from docx import Document

In [4]:
doc = Document(r'docx\Project_Report_Template.docx')

2. 遍历文档中的所有段落（Paragraph）和表格（Table）单元格，查找并替换 placeholder_map 中定义的所有占位符。
3. 替换占位符：
    - 若占位符出现在段落文本中，直接替换。
    - 若占位符出现在表格单元格中，需要先获取单元格的文本，替换后再设置回单元格。
4. 保留格式：在替换文本时，要注意保留原段落或单元格的格式，包括字体、大小、加粗、斜体等。

In [ ]:
for para in doc.paragraphs:
    for run in para.runs:
        if ''